In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import imageio
import scipy as sp

from torchvision import transforms

import torch
from mpl_toolkits import mplot3d

from scipy.optimize import curve_fit
from PIL import Image

from scipy.special import softmax
import imageio
#np.set_printoptions(precision=1)
%matplotlib notebook
#plt.close('all')
import networkx as nx
from sklearn.decomposition import PCA

In [2]:
def make_nodefeature(x,y,r,t):
    A=np.zeros((len(x),2))
       
    
    



def make_adjacency(x,y,r,t,ide,split_id,run):
    A=np.zeros((len(x),len(x)))
    x_id = []
    t_id = []
    for i in range(len(x)):
        x_id.append(i+1)
        t_id.append(t[i])
        for j in range(len(x)):
            if ide[i]==ide[j] and ide[i]!=-2:
                if t[i]+1==t[j] or t[i]-1==t[j]:
                    A[i,j]=1
            elif ide[i]==split_id[j]:
                if ide[i]==-1:
                    A[i,j]=1
                elif t[i]+1==t[j] or t[i]-1==t[j]:
                    A[i,j]=1
              
    
    
    np.savetxt('./'+str(run)+'_GT'+'/'+'A.txt', A,fmt='%i')
    np.savetxt('./'+str(run)+'/'+'timetable.txt',np.c_[x_id,t_id], fmt='%i',delimiter='\t')
    
    return A
        

    
def make_weighted_adj_estimate(x,y,t,run):
    
    #emb without void node
    
    x=x[1:]
    y=y[1:]
    t=t[1:]
    
    #############
    
    A=np.zeros((len(x),len(x)))
    
    for i in range(len(x)):
        for j in range(len(x)):
            #A[i,0]=1.5  ####
            if t[i]+1==t[j]: #t[i]+1==t[j] or
                A[i,j]=gaus2d(x[j], y[j], x[i], y[i]) #1/((x[i]-x[j])**2+(y[i]-y[j])**2)
            #if i>j:
                #A[i,j]=0
        if np.sum(A[i,:])!=0:
            A[i,:]=mean_normal(A[i,:])
        #print(A[i,:],i)
       
    A=A.round(decimals=2)
    np.fill_diagonal(A, 0.1)
    np.savetxt('./'+str(run)+'/'+'B.txt', A)
    return A
    
    
def make_dist_matrix(x,y,t,run):
    A=np.zeros((len(x),len(x)))
    
    for i in range(len(x)):
        for j in range(len(x)):
            if t[i]+1==t[j]: #t[i]+1==t[j] or
                A[i,j]=np.sqrt((x[i]-x[j])**2+(y[i]-y[j])**2)
       
    
    
    np.savetxt('./'+str(run)+'/'+'D.txt', A)
    return A


def make_weighted_edgelist(A,run):
    
    e1=[]
    e2=[]
    w=[]
    
    
    for i in range(len(A)):
        for j in range(len(A)):
            if i>j and j!=0:
                A[i,j]=0
            
            if A[i,j]!=0:
                e1.append(int(i+1))
                e2.append(int(j+1))
                w.append(A[i,j]*100)
    
    
    np.savetxt('./node2vec/graph/'+str(run)+'.edgelist', np.c_[e1,e2,w], fmt='%i',delimiter='\t')
    return A,e1,e2,w



def make_true_edgelist(A,run):
    
    e1=[]
    e2=[]
    w=[]
    
    
    for i in range(len(A)):
        for j in range(len(A)):
            if i>j:
                A[i,j]=0
            
            if A[i,j]!=0:
                e1.append(int(i+1))
                e2.append(int(j+1))
                #w.append(A[i,j])
    
    
    np.savetxt('./'+str(run)+'_GT'+'/'+'A.edgelist', np.c_[e1,e2], fmt='%i',delimiter='\t')
    return 0


def mean_normal(x):
    s=np.sum(x)
    return x #x/s


def gaus2d(x, y, mx, my, sx=0.07, sy=0.07):
    
    
    z=1. / (2. * np.pi * sx * sy) * np.exp(-((x - mx)**2. / (2. * sx**2.) + (y - my)**2. / (2. * sy**2.)))
    
    if z < 0.01:
        z=0.01
    
    
    
    return z



def padding(maxlen,run):
    #A,B,E
    
    B=np.loadtxt('./'+str(run)+'/'+'B.txt')
    A=np.loadtxt('./'+str(run)+'_GT'+'/'+'A.txt')
    E=np.loadtxt('./'+str(run)+'/'+'embed.txt')
    
    
    for i in range(len(A)):
        for j in range(len(A)):
            if i>j:
                A[i,j]=0
                B[i,j]=0
                
    np.savetxt('./'+str(run)+'_GT'+'/'+'A_padded.txt', A,fmt='%i')
    #A=np.zeros((4,4))
    #B=np.zeros((4,4))
    
   #E=np.ones((4,6))
    
    h=np.ones(maxlen)*-1
    v=np.ones(len(A[0]))*-1
    
    #
    cur_E=len(E)
    cur_A=len(A)
    cur_Ah=len(A[0])
    zero_line=np.zeros(len(E[0]))
    for i in range(maxlen-cur_E):
        E=np.vstack([E, zero_line])
        
    for j in range(maxlen-cur_A):
        A=np.vstack([A, v])
        B=np.vstack([B, v])
        
    for z in range(maxlen-cur_Ah):
        A=np.column_stack([A, h])
        B=np.column_stack([B, h])
        
    #np.savetxt('./'+str(run)+'_GT'+'/'+'A_padded.txt', A,fmt='%i')
    np.savetxt('./'+str(run)+'/'+'B_padded.txt', B)
    np.savetxt('./'+str(run)+'/'+'embed_padded.txt',E)
    print(A.shape)   
    return A,B,E    
    
#x = np.linspace(-5, 5)
#y = np.linspace(-5, 5)
#x, y = np.meshgrid(x, y) # get 2D variables instead of 1D
#z = gaus2d(x, y)

#def correct(A,B,E,max_len,run):



#def A_B_dense(A,B):

def fit_func(x,a,b):
    return a*x+b

def fingerprint_emb(pos_x,pos_y,window,t,run,void_node=True,a_x=113.89266922,b_x=19.51478307,a_y=-114.14194117,b_y=131.3404747):
    x_l,x_r,y_l,y_u=calibrate_pix_pos(pos_x,pos_y,a_x,b_x,a_y,b_y,window)
    im = blend(run,t)#Image.open(path).convert('L') # Can be many different formats.
    im_crop = im.crop((x_l, y_l, x_r, y_u))
     
    if t==9 and run==95:
        im_crop.show()
    foo = im_crop.resize((10,10),Image.ANTIALIAS)

    imarray = np.asarray(foo)
    imarray=imarray.flatten()/255
    return imarray



def fingerprint_wrap(window,run,void=False):
    pos_x,pos_y,t= np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t',usecols=(0,1,6), unpack=True)
    l=len(pos_x)-1
    
    
    f_e = np.array([np.zeros(100)])


    for k in range(int(np.max(t))+1):
        path='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(k)+'.png'
        pos_x_n=pos_x[t==k]
        pos_y_n=pos_y[t==k]
        if k==0:
            pos_x_n=pos_x_n[1:]
            pos_y_n=pos_y_n[1:]
        for j in range(len(pos_x_n)):
            imarray=norm_im_array(np.array([fingerprint_emb(pos_x_n[j],pos_y_n[j],window,k,run)]),offset=8.745098039215686292e-01)
            f_e=np.concatenate((f_e, imarray), axis=0)
            
    
    if void==False:
        f_e=f_e[1:]
            
        
    return f_e
    
    
def blend(run,t):
    
    
    
    pathm1='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(t-1)+'.png'
    path='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(t)+'.png'
    pathp1='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(t+1)+'.png'
    
    
    im = Image.open(path).convert('L')
    shape = np.asarray(im).shape
    
    #print(pixels.shape)
    try:
        imm1 = Image.open(pathm1).convert('L')
        pixels = np.asarray(imm1)
        #print(pixels)
        pixels = pixels.astype('float32')
        pixels = pixels/2
        #pixels = pixels.astype('int')
        imm1 = Image.fromarray(pixels).convert('L')
    except:
        pix=255*np.ones(shape)
        pix = pix.astype('float32')
        pix = pix/2
        imm1 = Image.fromarray(pix).convert('L')
        
    try:
        imp1 = Image.open(pathp1).convert('L')
    except:
        pix=255*np.ones(shape)
        imp1 = Image.fromarray(pix).convert('L')
    
    
    
    im3 = Image.blend(imm1, imp1, 0.5)
    im4 = Image.blend(im3, im, 0.5)
    #im4.show()
    return im4
    
    

def norm_im_array(arr,offset):
    return (arr-offset)*10
    
    
    
'''
pos_x=[5.926355409057428858e-01,4.604170197372819962e-01,6.250962551422760416e-02,1.223613241454796496e-01,6.376507269405902179e-01,1.103583087228645621e-01,4.233227679149828004e-01]
pos_y=[5.378065857159227692e-01,5.482527480620434934e-01,1.628245440499290486e-01,8.255699686855265673e-01,5.184511465256566032e-01,1.492480151352687523e-01,5.115442664966730613e-01]
pix_x=[87,72,27,33,92,32,68]
pix_y=[70,69,113,37,72,114,73]


plt.errorbar(pos_y, pix_y,fmt='.')   

popt, pcov = curve_fit(fit_func, pos_y,pix_y)

plt.plot(pos_y, fit_func(np.array(pos_y),*popt))
print(popt)
'''

a_y=-114.14194117
a_x=113.89266922
b_y=131.3404747
b_x=19.51478307
    
def calibrate_pix_pos(pos_x,pos_y,a_x,b_x,a_y,b_y,window):
    pix_x0=int(fit_func(pos_x,a_x,b_x))
    
    pix_y0=int(fit_func(pos_y,a_y,b_y))
    #print(pix_x0,pix_y0,int(pix_x0-window))
    return int(pix_x0-window),int(pix_x0+window),int(pix_y0-window),int(pix_y0+window)

def spectral_embedding(B,d,path=None):
    B=symmetrize_matrix(B)
    
    D=np.zeros(B.shape)
    
    column_sums = np.sum(B, axis=0)
    np.fill_diagonal(D, column_sums)
    
    L = D-B
    
    X = k_eigenvectors(L, d)
    
    #X=2*np.random.rand(X.shape[0],X.shape[1])-1
    #X=np.random.rand(X.shape[0],X.shape[1])
    X=RAPID_2(path,p_len=40,num_p=10,d=d)
    
    return X


def symmetrize_matrix(matrix):
    row, col = matrix.shape
    for i in range(row):
        for j in range(i, col):
            matrix[j][i] = matrix[i][j]
    return matrix    
    
    
def k_eigenvectors(matrix, k):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    eigenvectors = eigenvectors[:, np.argsort(eigenvalues)[::-1]]
    return eigenvectors[:, :k]    
    

    
def pos_enc(p,n):
    E=np.zeros((p,n))
    for k in range(int(p/2)):
        for i in range(n):
            E[2*k,i]=np.sin(2*i*np.exp(-(k*np.log(1000))/(p)))
            E[2*k+1,i]=np.cos(2*i*np.exp(-(k*np.log(1000))/(p)))
            
    return E

def fingerprint_cnn(pos_x,pos_y,window,t,run,void_node=False,a_x=113.89266922,b_x=19.51478307,a_y=-114.14194117,b_y=131.3404747):
    
    if void_node:
        pos_x = 0.74
        pos_y = 0.8
        x_l,x_r,y_l,y_u=calibrate_pix_pos(pos_x,pos_y,a_x,b_x,a_y,b_y,window)
        
        path='/home/mo/Desktop/IWR/TimeGraph2.0/void.png'
        
        im = Image.open(path).convert('L')
        shape = np.asarray(im).shape
        
        im_crop = im.crop((x_l, y_l, x_r, y_u))

        if t==9 and run==95:
            im_crop.show()


        im_crop = im_crop.resize((20,20),Image.ANTIALIAS)
        
        imarray = np.asarray(im_crop)
        

        im_tens=torch.zeros(3, 20,20)


        convert_tensor = transforms.ToTensor()
        im_tens[1]=convert_tensor(imarray)
        im_tens[0]=convert_tensor(imarray)
        im_tens[2]=convert_tensor(imarray)
        
    
    else:
        
        x_l,x_r,y_l,y_u=calibrate_pix_pos(pos_x,pos_y,a_x,b_x,a_y,b_y,window)


        pathm1='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(t-1)+'.png'
        path='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(t)+'.png'
        pathp1='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(t+1)+'.png'


        im = Image.open(path).convert('L')
        shape = np.asarray(im).shape
        try:
            im_m = Image.open(pathm1).convert('L')
        except:
            pix=255*np.ones(shape)
            im_m = Image.fromarray(pix).convert('L')
        try:
            im_p = Image.open(pathp1).convert('L')
        except:
            pix=255*np.ones(shape)
            im_p = Image.fromarray(pix).convert('L')





        im_crop = im.crop((x_l, y_l, x_r, y_u))
        im_m_crop = im_m.crop((x_l, y_l, x_r, y_u))
        im_p_crop = im_p.crop((x_l, y_l, x_r, y_u))






        if t==9 and run==95:
            im_crop.show()


        im_crop = im_crop.resize((20,20),Image.ANTIALIAS)
        im_m_crop = im_m_crop.resize((20,20),Image.ANTIALIAS)
        im_p_crop = im_p_crop.resize((20,20),Image.ANTIALIAS)

        imarray = np.asarray(im_crop)
        print('shape',imarray.shape)
        im_m_array = np.asarray(im_m_crop)
        im_p_array = np.asarray(im_p_crop)


        im_tens=torch.zeros(3, 20,20)


        convert_tensor = transforms.ToTensor()
        im_tens[1]=convert_tensor(imarray)
        im_tens[0]=convert_tensor(im_m_array)
        im_tens[2]=convert_tensor(im_p_array)

    return im_tens

def fingerprint_cnn_wrap(window,run,void=False):
    pos_x,pos_y,t= np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t',usecols=(0,1,6), unpack=True)
    
    
    
    



    for k in range(int(np.max(t))+1):
        path='/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(k)+'.png'
        pos_x_n=pos_x[t==k]
        pos_y_n=pos_y[t==k]
        void_tens=fingerprint_cnn(0,0,window,k,run,void_node=True)
        tensor_list = [void_tens]
        if k==0:
            pos_x_n=pos_x_n[1:]
            pos_y_n=pos_y_n[1:]
        for j in range(len(pos_x_n)):
            im_tens=fingerprint_cnn(pos_x_n[j],pos_y_n[j],window,k,run)
            tensor_list.append(im_tens)
            
        #void_tens=fingerprint_cnn(pos_x_n[j],pos_y_n[j],window,k,run,void_node=True)
        #tensor_list.append(void_tens)    
        stacked_tensor = torch.stack(tensor_list)
        
        
        torch.save(stacked_tensor, '/home/mo/Desktop/IWR/TimeGraph2.0/'+str(run)+'/'+str(k)+'.pt')

        

def add_dicts(dict1,dict2):

    for key in dict2:
        if key in dict1:
            dict1[key] += dict2[key]
        else:
            dict1[key] = dict2[key]
            
    return dict1

        
        
def edge_list_conv(path):
    edge_list = []
    max_n = [1]

    with open(path, 'r') as f:
        for line in f:
            parts = line.split()
            edge_list.append((int(parts[0]), int(parts[1]), int(parts[2])))
            #edge_list.append((int(parts[1]), int(parts[0]), int(parts[2])))
            max_n.append(int(parts[1]))
    #print(max_n)
    num_nodes=np.max(max_n)
    return edge_list,num_nodes

def RAPID(path_start,p_len,num_p,iters,d=50):
    # Create a weighted graph
    #d=50
    #p_len=10
    G = nx.Graph()
    #path_start='/home/mo/Desktop/IWR/TimeGraph2.0/node2vec/graph/7.edgelist'
    edge_list,num_nodes=edge_list_conv(path_start)
    
    G.add_weighted_edges_from(edge_list)
    
    #f_mat=np.random.rand(num_nodes, d)
    
    #f_res=np.random.rand(num_nodes, d)
    f_res= np.random.uniform(-1, 1, size=(num_nodes,d))
    for g in range(iters):
        f_mat=f_res
        for j in range(1,num_nodes+1):
            #f_mat=f_res
            count_tot = {}
            for k in range(num_p):


                # Perform a random walk with restart from node 1
                start_node = j
                current_node = start_node
                path = [current_node]

                for i in range(p_len):
                    #print('c',current_node)
                    # Compute the probabilities of moving to neighboring nodes
                    #probs = np.zeros(len(G.nodes))
                    probs=[]
                    for neighbor in G.neighbors(current_node):
                        #print(neighbor)
                        #probs[neighbor-1] = G[current_node][neighbor]['weight']
                        probs.append(G[current_node][neighbor]['weight'])

                    # Normalize the probabilities
                    probs /= np.sum(probs)

                    #print(probs)

                    # Choose the next node to visit based on the probabilities
                    current_node = np.random.choice(list(G.neighbors(current_node)), p=probs)

                    # Add the node to the path
                    path.append(current_node)

                #print(path)
                counts = {}

                for item in path:
                    if item in counts:
                        counts[item] += 1
                    else:
                        counts[item] = 1
                add_dicts(count_tot,counts)
                #print('tot',j,k,count_tot)


            arrays = [value * f_mat[key-1] for key, value in count_tot.items()]
            #print(arrays)
            # sum up the arrays to get the resulting array
            result = np.sum(arrays, axis=0)
            f_res[j-1]=result/(num_p*p_len)
        
    return f_res


def RAPID_2(path_start,p_len,num_p,d=50):
    # Create a weighted graph
    #d=50
    #p_len=10
    G = nx.Graph()
    #path_start='/home/mo/Desktop/IWR/TimeGraph2.0/node2vec/graph/7.edgelist'
    edge_list,num_nodes=edge_list_conv(path_start)
    
    G.add_weighted_edges_from(edge_list)
    
    #f_mat=np.random.rand(num_nodes, d)
    
    f_res=np.zeros((num_nodes, num_nodes))
    #f_res= np.random.uniform(-1, 1, size=(num_nodes,d))
    
    for j in range(1,num_nodes+1):
        
        #f_mat=f_res
        count_tot = {}
        for k in range(num_p):


            # Perform a random walk with restart from node 1
            start_node = j
            current_node = start_node
            path = [current_node]

            for i in range(p_len):
                #print('c',current_node)
                # Compute the probabilities of moving to neighboring nodes
                #probs = np.zeros(len(G.nodes))
                probs=[]
                for neighbor in G.neighbors(current_node):
                    #print(neighbor)
                    #probs[neighbor-1] = G[current_node][neighbor]['weight']
                    probs.append(G[current_node][neighbor]['weight'])

                # Normalize the probabilities
                probs /= np.sum(probs)

                #print(probs)

                # Choose the next node to visit based on the probabilities
                current_node = np.random.choice(list(G.neighbors(current_node)), p=probs)

                # Add the node to the path
                path.append(current_node)

            #print(path)
            counts = {}

            for item in path:
                if item in counts:
                    counts[item] += 1
                else:
                    counts[item] = 1
            add_dicts(count_tot,counts)
            #print('tot',j,k,count_tot)
        #print(j)
        #print('tot',count_tot)
        #arrays = [value * f_mat[key-1] for key, value in count_tot.items()]
        #print(arrays)
        # sum up the arrays to get the resulting array
        #result = np.sum(arrays, axis=0)
        for key, value in count_tot.items():
            f_res[j-1,key-1]=value
    pca = PCA(n_components=d)
    f_fin=pca.fit_transform(f_res)
    norms = np.linalg.norm(f_fin, axis=1, keepdims=True)
    f_fin = f_fin / norms
    return f_fin


#(Random-wAlk-based Path-feature calculatIon on Weighted graphs)




In [3]:
'''
import numpy as np

two iteration?
1st weights 
2nd similarity as weight?


'''
print(stop)
d=2
p_len=40
iters=1
path_start='/home/mo/Desktop/IWR/TimeGraph2.0/node2vec/graph/7.edgelist'
num_p=15
f_res=RAPID_2(path_start,p_len,num_p,d)

print(np.linalg.norm(f_res[0]))
print(f_res[3])

plt.scatter(f_res[:, 0], f_res[:, 1])

plt.scatter(f_res[0, 0], f_res[0, 1],c='red')

plt.scatter(f_res[3, 0], f_res[3, 1],c='red')
plt.scatter(f_res[7, 0], f_res[7, 1],c='red')


plt.scatter(f_res[1, 0], f_res[1, 1],c='green')

plt.scatter(f_res[5, 0], f_res[5, 1],c='green')
plt.scatter(f_res[8, 0], f_res[8, 1],c='green')


plt.scatter(f_res[14, 0], f_res[14, 1],c='blue')

plt.scatter(f_res[9, 0], f_res[9, 1],c='purple')
plt.scatter(f_res[10, 0], f_res[10, 1],c='purple')
plt.scatter(f_res[16, 0], f_res[16, 1],c='purple')
plt.scatter(f_res[78, 0], f_res[78, 1],c='purple')

NameError: name 'stop' is not defined

In [ ]:
A=np.loadtxt('./'+str(2)+'/'+'embed.txt')
print(A)

A=np.vstack([A, np.zeros(len(A[0]))])
#A=np.append(A,np.zeros(len(A[0])))
#print(A)


A=np.ones((3,4))
B=np.zeros((3,2))
print(np.c_[A, B])
print(gaus2d(1.8*0.07, 1.8*0.07, 0, 0, sx=0.07, sy=0.07))
C=np.zeros((3,3))
C[0,1]=0.3
C[0,2]=0.3
C[2,2]=0.7

print(C)
L=spectral_embedding(C,2)
print(L)
#print(k_eigenvectors(L, 2))

#A,B,E =padding(7,1)

#print(E)
#blend(10)

In [4]:
lenA=[]
for i in range(1,100):

    run=i

    x,y,r,ide,split_id,split_prob,t= np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t', usecols=(0,1,2,3,4,5,6), unpack=True)
    #print(len(x))
    A=make_adjacency(x,y,r,t,ide,split_id,run)
    print(len(A),i)
    lenA.append(len(A))
    
    B=make_weighted_adj_estimate(x,y,t,run)
    #print(B)
    C,e1,e2,w=make_weighted_edgelist(B,run)
    
    
    
    
    path_start='/home/mo/Desktop/IWR/TimeGraph2.0/node2vec/graph/'+str(run)+'.edgelist'
    s=spectral_embedding(B,path=path_start,d=60)
    s=np.concatenate((np.array([np.zeros(len(s[0]))]), s), axis=0)
    
    #s=np.concatenate((np.array([2*np.random.rand(len(s[0]))-1]), s), axis=0)
    np.savetxt('./'+str(run)+'/'+'embed.txt', s )
    
    make_true_edgelist(A,run)
    
    make_dist_matrix(x,y,t,run)
    
maxlen=np.max(lenA)
print(maxlen)






227 1
223 2
203 3
265 4
230 5
227 6
233 7
221 8
230 9
192 10
171 11
232 12
221 13
244 14
222 15
203 16
222 17
234 18
199 19
250 20
273 21
226 22
274 23
190 24
210 25
205 26
233 27
223 28
227 29
208 30
228 31
211 32
217 33
218 34
222 35
307 36
214 37
290 38
224 39
255 40
214 41
218 42
237 43
256 44
230 45
210 46
239 47
235 48
277 49
238 50
282 51
235 52
199 53
234 54
247 55
223 56
243 57
279 58
223 59
232 60
230 61
224 62
230 63
225 64
220 65
206 66
196 67
205 68
236 69
291 70
231 71
262 72
240 73
242 74
231 75
241 76
223 77
256 78
225 79
218 80
243 81
245 82
215 83
225 84
283 85
240 86
229 87
234 88
237 89
245 90
224 91
234 92
250 93
264 94
207 95
253 96
237 97
198 98
205 99
307


In [5]:
#print(stop)

for i in range(1,100):

    run=i

    b = np.loadtxt('./node2vec/emb/'+str(run)+'.emb',skiprows=1, delimiter=' ', unpack=True)
    

    ind=np.argsort(b[0])

    for l in range(len(b)):
        b[l]= np.take_along_axis(b[l], ind,axis=0)  
    b=np.transpose(b)
    b=b[:,1:]
    
    
    ####
    b=np.concatenate((np.array([np.zeros(len(b[0]))]), b), axis=0) #######if void node not included in n2v
    ####
    
    
    fingerprint_cnn_wrap(20,run,void=False)  #always 000000000000000 for fingerprint
    #b=np.c_[b, f_e]
    #print(len(f_e),run)
    
    #pos_x,pos_y= np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t',, unpack=True)
    #id,tt = np.loadtxt('./'+str(run)+'/'+'timetable.txt', delimiter='\t', usecols=(0,1), unpack=True)
    #split_prob = pos.T[:,5]
    #split_prob = split_prob[:, np.newaxis]
    #r = pos.T[:,2]
    #r=r[:, np.newaxis]
    
    #print(b.shape,pos.shape)
    
    #b=np.append(b,r,axis=1)
    #b=np.append(b,split_prob,axis=1)
    
    #np.savetxt('./'+str(run)+'/'+'embed.txt', b)
    
    
    ##remove below if n2v is used
    
    print(len(b[0]),run)
    
    #p = np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t', unpack=True)
    #split_prob = p.T[:,5]
    #print(len(split_prob),split_prob)
    #b = b.T[:,0:3]
    #split_prob=split_prob[:, np.newaxis]
    #print(len(b),b)
    
    #print(b.shape,split_prob.shape)
    
    #######b=np.append(b,split_prob,axis=1)
    
    #b=np.append(pos,split_prob,axis=1)
    
    
    #print(b)
    #b=np.repeat(b, 6, axis=1)
    #print(i)
    #print(b)
    
    
    
    #np.savetxt('./'+str(run)+'/'+'embed_n2v.txt', b)
    #padding(maxlen,run)
    

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)


/home/mo/anaconda3/lib/python3.7/site-packages/torchvision/transforms/functional.py:150: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20,

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
50 3
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20,

...

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
50 87
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shap

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20,

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
50 92
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shap

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20,

shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20, 20)
shape (20,

python ./src/main.py --input ./graph/14.edgelist --output ./emb/14.emb --dimensions 25 --walk-length 40 --num-walks 300 --window-size 30 --p 0.25 --q 4 --iter 20 --weighted

In [ ]:

#np.savetxt('./node2vec/emb/'+str(10)+'.emb',i)
x,y= np.loadtxt('./'+str(2)+'/'+'embed.txt',usecols=(0,1),unpack=True)
print(x,y)
#i=ide
#print(i)
#print(i[0])
plt.scatter(x,y,s=1)
#plt.scatter(x[1],y[1],c='r')
#plt.scatter(x[i==4],y[i==4],c='blue')
#plt.scatter(x[i==5],y[i==5],c='g')
#plt.scatter(x[i==6],y[i==6],c='black')
#plt.scatter(x[i==7],y[i==7],c='violet')
#plt.scatter(x[i==10],y[i==10],c='orange')
#plt.scatter(x[i==2],y[i==2],c='grey')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

run=85

x,y,r,ide,split_id,split_prob,t= np.loadtxt('./'+str(run)+'_GT'+'/'+'pos_GT.txt',skiprows=1, delimiter='\t', usecols=(0,1,2,3,4,5,6), unpack=True)



x=x[1:]
y=y[1:]
t=t[1:]

id1,id2=np.loadtxt('./'+str(run)+'_GT'+'/'+'reconstruct.edgelist', delimiter='\t', usecols=(0,1), unpack=True)
#id1,id2=np.loadtxt('./'+str(run)+'_GT'+'/'+'A.edgelist', delimiter='\t', usecols=(0,1), unpack=True)
#id1,id2,w=np.loadtxt('./node2vec/graph/'+str(run)+'.edgelist', delimiter='\t', usecols=(0,1,2), unpack=True)
#print(len(id1))
id2=id2[id1!=1]
id1=id1[id1!=1]

#w=w/np.max(w)


xline=[]
yline=[]
tline=[]
for i in range(len(id1)):
    
    xline=[]
    yline=[]
    tline=[]
    
    print(id1[i],id2[i])
    xline.append(x[int(id1[i]-1)])   
    yline.append(y[int(id1[i]-1)])
    tline.append(t[int(id1[i]-1)])
    
    xline.append(x[int(id2[i]-1)])
    yline.append(y[int(id2[i]-1)])
    tline.append(t[int(id2[i]-1)])
    
    print(xline)
    
    #ax.plot3D(tline, yline, xline, c='red',alpha=w[i])
    ax.plot3D(tline, yline, xline, c='grey')


#zline = np.linspace(0, 15, 1000)
#xline = np.sin(zline)
#yline = np.cos(zline)
#

# Data for three-dimensional scattered points
zdata = x
xdata = t
ydata = y
ax.scatter3D(xdata, ydata, zdata);

#plt.savefig('./TimeGraph.png',dpi=my_dpi,transparent=False) 

In [ ]:
def colour_func(ii,prev_c,prev_id,id1,id2):
    id1=id1[id2==ii]
    #print(id1,id2)
    if len(id1)==0:
        print('ii',ii)
        print(prev_c)
        return prev_c[int(ii-2)],False
    if id1==1:
        return 'black',True
    else:
        #print(id1,prev_id,prev_c)
        print('prev_c',np.array(prev_c))
        c_cf=np.array(prev_c)[prev_id==id1[0]]
        print('c',c_cf)
    return c_cf[0],False
    


c=['blue','green','violet']

id,tt = np.loadtxt('./'+str(run)+'/'+'timetable.txt', delimiter='\t', usecols=(0,1), unpack=True)
id=id[1:]
tt=tt[1:]
id_f=[2,3,4]


print(c[id_f==1])


    


for t in range(31):
    
        
    
    my_dpi=96
    pix=150
    
    
    fig = plt.figure(figsize=(pix/my_dpi, pix/my_dpi), dpi=my_dpi)
    fig = plt.gcf()
    ax = fig.gca()
    
    print('c_full',c)
    prev_c=c
    prev_id=id_f
    
    c=[]
    id_f=[]
    
    for u in range(len(id[tt==t])):
        x_n=x[int(id[tt==t][u]-1)]  
        y_n=y[int(id[tt==t][u]-1)]
        r_n=r[int(id[tt==t][u]-1)]  
        c_cf,rec=colour_func(id[tt==t][u],prev_c,prev_id,id1,id2)
        c.append(c_cf)
        #print('c.app',c)
        id_f.append(id[tt==t][u])
        if rec:
            rec1=plt.Rectangle((x_n,y_n), r_n, r_n,fc=c_cf,ec=c_cf) 
            ax.add_patch(rec1)
        else:
            circ1=plt.Circle((x_n,y_n), r_n, color=c_cf)
            ax.add_patch(circ1)
        
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis('off')
    plt.xlim(0,1)
    plt.ylim(0,1)
    
    plt.savefig('./'+str(run)+'_GT'+'/'+str(t)+'.png',dpi=my_dpi,transparent=False) 

In [ ]:
x_t=[]
for j in range(len(ide)):
    x_t=[]
    x_t.append(x[ide==j])
    print(str(j),x_t)